In [1]:
# normalize a circuit from QpenQASM 3.0 to Quartz readable qasm
# Gate set: rigetti

import os
from os.path import join, isfile

In [2]:
qasm_path = os.getcwd() + '/rigetti_circs/'
qasm_fns = [fn for fn in os.listdir(qasm_path) if isfile(join(qasm_path, fn)) and fn[-4:] == 'qasm' and fn[0] != '_']
qasm_full_paths = [qasm_path + fn for fn in qasm_fns]

In [3]:
for qasm_fn in qasm_full_paths:
    with open(qasm_fn, 'r') as f:
        text: str = f.read()
        lines: list[str] = text.split('\n')
        lines = [line for line in lines if line != '' and line[:2] != '//']
        lines = [line for line in lines if line[:7] != 'barrier' and line[:7] != 'measure']
        lines = [line for line in lines if line[:7] != 'rz(0.0)']

        new_lines: list[str] = []
        for line in lines:
            line += '\n'
            if line[:2] == "rx":
                if line[:8] == "rx(pi/2)":
                    new_lines.append(line.replace("rx(pi/2)", "rx1"))
                elif line[:6] == "rx(pi)":
                    new_lines.append(line.replace("rx(pi)", "x"))
            else:
                new_lines.append(line)

    with open(f"{qasm_fn[:-5]}_norm.qasm", 'w') as f:
        f.writelines(new_lines)

In [5]:
import quartz

quartz_context = quartz.QuartzContext(gate_set=['cz', 'x', 'rz', 'add', 'rx1', 'rx3'], filename='ecc_set/rigetti_4_ecc.json')
for qasm_fn in qasm_full_paths:
    print(qasm_fn)
    if qasm_fn.find("norm") == -1:
        continue
    circ = quartz.PyGraph.from_qasm(context=quartz_context, filename=qasm_fn)
    print(circ.to_qasm_str())

: 

: 